In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "GPU(s) física(s),", len(logical_gpus), "GPU(s) lógica(s)")
        print("Funciona")
    except RuntimeError as e:
        print(e)
else:
    print("Error")

1 GPU(s) física(s), 1 GPU(s) lógica(s)
Funciona


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cv2 import CascadeClassifier
import os
import re
import tensorflow
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model

import tensorflow_hub as hub


import bert
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

import tensorflow as tf
from keras.models import load_model
import cv2
from PIL import Image
from transformers import TFRobertaModel, AutoTokenizer
import numpy as np
import re
import pandas as pd


In [3]:
import random, os
import numpy as np
#import tensorflow as tf

def set_seed(seed=42):
  np.random.seed(seed) 
  tf.random.set_seed(seed) 
  random.seed(seed)
  os.environ['TF_DETERMINISTIC_OPS'] = "1"
  os.environ['TF_CUDNN_DETERMINISM'] = "1"
  os.environ['PYTHONHASHSEED'] = str(seed)

set_seed()

In [37]:
def custom_objects_fn():
    return {'TFRobertaModel': TFRobertaModel}

modeltext = load_model('BERT_CNN_model_best_13_val_0.7836.h5', custom_objects=custom_objects_fn())
print("Modelo cargado text")
modelfacial = load_model('best_model_ModelPaper1_200_7020Test.h5')
print("Modelo cargado facial")

Modelo cargado text
Modelo cargado facial


In [45]:
image_input = tf.keras.layers.Input(shape=(48,48,3), name='image_input')
text_input_ids = tf.keras.layers.Input(shape=(300,), dtype=tf.int32, name='input_ids')
text_attention_mask = tf.keras.layers.Input(shape=(300,), dtype=tf.int32, name='attention_mask')

features_facial = modelfacial(image_input)
features_text = modeltext([text_input_ids,text_attention_mask])
merged = tf.keras.layers.Concatenate()([features_facial, features_text])

multimodal_model = tf.keras.models.Model(inputs=[image_input,text_input_ids,text_attention_mask], outputs=merged)
multimodal_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

multimodal_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 300)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 300)]        0           []                               
                                                                                                  
 model_42 (Functional)          (None, 7)            43426146    ['image_input[0][0]']            
                                                                                            

In [46]:
from transformers import RobertaTokenizer, RobertaModel
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = RobertaModel.from_pretrained("FacebookAI/roberta-base")

def prepare_text_data(texts):
    inputs = tokenizer(texts, return_tensors='tf', padding='max_length', truncation=True, max_length=300)
    input_ids = inputs['input_ids']
    attention_masks = inputs['attention_mask']
    return input_ids, attention_masks

def prepare_image_data(image_filepath):
    image_data = []
    image = Image.open(image_filepath).convert('L')
    image = image.resize((48, 48))
    image = np.array(image) / 255.0  
    image = np.stack((image,)*3, axis=-1)
    image_data.append(image)
    return np.array(image_data)

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
def predict_multimodal(text, image_filepaths):
    text_data = text
    text_input_ids, text_attention_masks = prepare_text_data(text_data)

    image_data = image_filepaths
    image_inputs = prepare_image_data(image_data)

    predictions = multimodal_model.predict([image_inputs, text_input_ids, text_attention_masks])
    return predictions

texts = "I think is lineal model"
image_filepaths = "test.png"

predictions = predict_multimodal(texts, image_filepaths)
clases = ["enojado", "disgusto", "asustado", "feliz", "neutral", "triste", "sorprendido"]
predictions_indice = np.argmax(predictions)
emotion_final = clases[predictions_indice]
print(emotion_final)


1/1 [==============================] - 0s 108ms/step
neutral
